In [ ]:
# | default_exp cli.api_key

In [ ]:
from airt.testing import activate_by_import

[INFO] airt.testing.activate_by_import: Testing environment activated.


In [ ]:
# | export

from typing import *

In [ ]:
# | exporti

import os

import pandas as pd
import typer
from tabulate import tabulate
from typer import echo

from airt.cli import helper
from airt.client import Client
from airt.constant import CLIENT_NAME, SERVICE_TOKEN
from airt.logger import get_logger, set_level

In [ ]:
import logging
import random
import string
from contextlib import contextmanager

import pytest
from typer.testing import CliRunner

import airt.sanitizer
from airt.cli.user import app as user_cli_app
from airt.constant import SERVICE_PASSWORD, SERVICE_TOKEN, SERVICE_USERNAME

In [ ]:
RANDOM_UUID_FOR_TESTING = "00000000-0000-0000-0000-000000000000"

In [ ]:
# | exporti

app = typer.Typer(
    help=f"""A set of commands for managing the ApiKeys in the server.
        
        Both the ApiKey and the token can be used for accessing the {CLIENT_NAME} services. However, there is a 
        slight difference in generating and managing the two.
        
        For generating the ApiKey, you first need to get the developer token. Please refer to **{CLIENT_NAME} token** command documentation to generate one.
        
        After logging in with your developer token, you can create any number of new ApiKeys and can set an 
        expiration date individually. You can also access other commands available as part of **{CLIENT_NAME} api-key** sub-command to 
        list, revoke the ApiKeys at any time.

        Once the new API key is generated, please set it in the **{SERVICE_TOKEN}** environment variable to start accessing the {CLIENT_NAME} services with it.""",
)

In [ ]:
assert "**AIRT_SERVICE_TOKEN**" in app.info.help
assert "airt services with it" in app.info.help
assert "**airt api-key**" in app.info.help

app.info.help

'A set of commands for managing the ApiKeys in the server.\n        \n        Both the ApiKey and the token can be used for accessing the airt services. However, there is a \n        slight difference in generating and managing the two.\n        \n        For generating the ApiKey, you first need to get the developer token. Please refer to **airt token** command documentation to generate one.\n        \n        After logging in with your developer token, you can create any number of new ApiKeys and can set an \n        expiration date individually. You can also access other commands available as part of **airt api-key** sub-command to \n        list, revoke the ApiKeys at any time.\n\n        Once the new API key is generated, please set it in the **AIRT_SERVICE_TOKEN** environment variable to start accessing the airt services with it.'

In [ ]:
runner = CliRunner()

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
set_level(logging.WARNING)

In [ ]:
# Testing logger settings

display(logger.getEffectiveLevel())
assert logger.getEffectiveLevel() == logging.WARNING

logger.debug("This is a debug message")
logger.info("This is an info")
logger.warning("This is a warning")
logger.error("This is an error")

30

[WARNING] __main__: This is a warning
[ERROR] __main__: This is an error


In [ ]:
# Helper context manager for testing

_airt_service_token = None


@contextmanager
def set_airt_service_token_envvar():
    global _airt_service_token
    if _airt_service_token is None:
        display("_airt_service_token is None, getting a token...")

        username = os.environ[SERVICE_USERNAME]
        password = os.environ[SERVICE_PASSWORD]

        Client.get_token(username=username, password=password)
        _airt_service_token = Client.auth_token

    try:
        os.environ[SERVICE_TOKEN] = _airt_service_token

        yield
    finally:
        del os.environ[SERVICE_TOKEN]

In [ ]:
with set_airt_service_token_envvar():
    display("*" * len((os.environ[SERVICE_TOKEN])))

'_airt_service_token is None, getting a token...'

'*******************************************************************************************************************************'

In [ ]:
# | exporti


@app.command()
@helper.requires_totp()
@helper.requires_auth_token
def create(
    name: str = typer.Argument(..., help="The name of the ApiKey."),
    expiry: int = typer.Option(
        None,
        "--expiry",
        "-e",
        help="The validity of the API key in number of days. If not passed, then the default value None will be used to create an ApiKey with no expiry date!",
    ),
    otp: Optional[str] = typer.Option(
        None,
        "--otp",
        help="Dynamically generated six-digit verification code from the authenticator app. Please pass this optional argument only if the MFA is enabled for your account.",
    ),
    quiet: bool = typer.Option(
        False,
        "--quiet",
        "-q",
        help="Output access token only.",
    ),
    debug: bool = typer.Option(
        False,
        "--debug",
        "-d",
        help="Set logger level to DEBUG and output everything.",
    ),
) -> None:
    """Create a new ApiKey

    !!! note

        - The name of the ApiKey must be unique. If not, an exception will be raised while creating a new key with an existing key's name.

        - The expiry for an ApiKey is optional. If not passed, then the default value None will be used to create an ApiKey with no expiry date!
    """

    from airt.client import APIKey

    response = APIKey.create(name=name, expiry=expiry, otp=otp)

    if quiet:
        typer.echo(response["access_token"])
    else:
        typer.echo(f"Access Token: {response['access_token']}")

In [ ]:
# Tests for APIKey create

# Testing negative scenario

result = runner.invoke(
    app,
    ["test", "-e", 365],
)

display(result.stdout)
assert f"KeyError: The environment variable '{SERVICE_TOKEN}' is not set." in str(
    result.stdout
)

"KeyError: The environment variable 'AIRT_SERVICE_TOKEN' is not set.\n\nPlease run the command 'airt token' to get the application token and set it in the environment variable `AIRT_SERVICE_TOKEN`.\n\nTry 'airt token --help' for help.\n"

In [ ]:
def generate_random_name(size=15, chars=string.ascii_uppercase + string.digits):
    return "".join(random.choice(chars) for _ in range(size))


assert len(generate_random_name()) == 15
assert type(generate_random_name()) == str

In [ ]:
# Tests for APIKey create

# Testing positive scenario and negative scenario


def mask(s: str) -> str:
    return "*" * len(s)


key_name = generate_random_name()
with set_airt_service_token_envvar():

    # Testing without quite flag
    result = runner.invoke(
        app,
        [key_name, "-e", 365],
    )

    access_token = result.stdout[:-1]
    masked_token = mask(access_token.split(": ")[1])

    display(f"{masked_token=}")

    assert result.exit_code == 0
    assert len(access_token) >= 127  # maybe
    assert "Access Token" in str(result.stdout)

    # Testing with quite flag
    result = runner.invoke(
        app,
        [generate_random_name(), "-e", 365, "-q"],
    )

    access_token = result.stdout[:-1]
    masked_token = mask(access_token)

    display(f"{masked_token=}")

    assert result.exit_code == 0
    assert len(access_token) >= 127  # maybe
    assert "Access Token" not in str(result.stdout)

    # Negative scenario: Creating key without setting expiration date
    result = runner.invoke(
        app,
        [generate_random_name(), "-q"],
    )

    access_token = result.stdout[:-1]
    masked_token = mask(access_token)
    display(f"{masked_token=}")

    assert result.exit_code == 0
    assert len(access_token) >= 110, len(access_token)  # maybe
    assert "Access Token" not in str(result.stdout)

    # Negative scenario: Passing existing key name
    result = runner.invoke(
        app,
        [key_name, "-e", 365, "-q"],
    )

    assert result.exit_code == 1
    assert "An Api-key with the same name already exists" in str(result.stdout)
    display(result.stdout)

    # Negative scenario: Passing OTP for non-mfa user existing key name
    random_otp = "123456"
    result = runner.invoke(
        app,
        [generate_random_name(), "-e", 365, "-q", "--otp", random_otp],
    )

    assert result.exit_code == 1
    assert "MFA is not activated for the account" in str(result.stdout)
    display(result.stdout)

"masked_token='*************************************************************************************************************************************************************************************************'"

"masked_token='*************************************************************************************************************************************************************************************************'"

"masked_token='***************************************************************************************************************************************************************************'"

'Error: An Api-key with the same name already exists\n'

'Error: MFA is not activated for the account. Please pass the OTP only after activating the MFA for your account.\n'

In [ ]:
# | exporti


@app.command()
@helper.display_formated_table
@helper.requires_auth_token
def ls(
    user: Optional[str] = typer.Option(
        None,
        "--user",
        "-u",
        help=f"user_uuid/username associated with the ApiKey. To get the user account uuid/username, use the `{CLIENT_NAME} user details` command."
        f" If the user_uuid/username is not passed, then the currently logged-in user_uuid/username will be used.",
    ),
    offset: int = typer.Option(
        0,
        "--offset",
        "-o",
        help="The number of ApiKeys to offset at the beginning. If None, then the default value 0 will be used.",
    ),
    limit: int = typer.Option(
        100,
        "--limit",
        "-l",
        help="The maximum number of ApiKeys to return from the server. If None, then the default value 100 will be used.",
    ),
    include_disabled: bool = typer.Option(
        False,
        "--disabled",
        help="If set to **True**, then the disabled ApiKeys will also be included in the result.",
    ),
    format: Optional[str] = typer.Option(
        None,
        "--format",
        "-f",
        help="Format output and show only the given column(s) values.",
    ),
    quiet: bool = typer.Option(
        False,
        "--quiet",
        "-q",
        help="Output only ApiKey uuids.",
    ),
    debug: bool = typer.Option(
        False,
        "--debug",
        "-d",
        help="Set logger level to DEBUG and output everything.",
    ),
) -> Dict["str", Union[pd.DataFrame, str]]:
    """Get the list of ApiKeys."""
    from airt.client import APIKey

    response = APIKey.ls(
        user=user, offset=offset, limit=limit, include_disabled=include_disabled
    )
    df = APIKey.as_df(response)

    df["created"] = helper.humanize_date(df["created"])
    df["expiry"] = helper.humanize_date(df["expiry"])

    return {"df": df}

In [ ]:
# Tests for APIKey ls

# Testing positive scenario
def get_values_from_result(result) -> List[Union[int, str]]:
    return [val for val in result.stdout[:-1].split("\n")]


with set_airt_service_token_envvar():

    # Testing without quite flag
    result = runner.invoke(app, ["ls"])
    display(result.stdout)

    assert result.exit_code == 0

    # With quiet
    result = runner.invoke(app, ["ls", "-q"])
    display(result.stdout)

    assert result.exit_code == 0
    ids = get_values_from_result(result)
    assert len(ids) > 0

'uuid                                  name             created         expiry              disabled\n76a90be6-4535-4c70-ac2a-16df432913ca  RKO4OSLE9DITOFO  24 minutes ago  None                False\n35a317ad-0fa1-4e76-ab36-d60a9a31c235  7XIE331PO1OWE71  24 minutes ago  5 months ago        False\n6146ab3a-a49b-4ae3-a628-b6a31583b2d2  DNAK3ICMGDAFQ1K  24 minutes ago  None                False\n3cc3ff1c-374a-4090-bb78-742e5b3aef0a  8TMEGHX4WXTDGJZ  24 minutes ago  None                False\n510edef2-244c-49c8-b8f5-b2c3b500c51b  YT9B6CT1797HNYP  24 minutes ago  None                False\ne4946c54-756e-4e74-9db5-22270bd1c025  SFFJK74O7N32MOR  24 minutes ago  None                False\n4313caa8-8faa-42ab-8f46-bd6669d96aad  IBK7RGHPPG05S4X  24 minutes ago  None                False\n6ad2a881-bfad-4acb-9d0b-13650e161051  R9JBTBYQF74FSVT  24 minutes ago  None                False\na7bfc801-649e-4574-997f-0dc57bd4e8b5  2BO0JKANPJ46HID  24 minutes ago  None                False\n00d6b61a-62f9-42

'76a90be6-4535-4c70-ac2a-16df432913ca\n35a317ad-0fa1-4e76-ab36-d60a9a31c235\n6146ab3a-a49b-4ae3-a628-b6a31583b2d2\n3cc3ff1c-374a-4090-bb78-742e5b3aef0a\n510edef2-244c-49c8-b8f5-b2c3b500c51b\ne4946c54-756e-4e74-9db5-22270bd1c025\n4313caa8-8faa-42ab-8f46-bd6669d96aad\n6ad2a881-bfad-4acb-9d0b-13650e161051\na7bfc801-649e-4574-997f-0dc57bd4e8b5\n00d6b61a-62f9-425a-a514-6383b9ffb392\n6e68750d-7c2b-49f4-b2fb-0c3795ec4a32\n78837644-436a-4e4f-82d0-c875d8a72437\n835281e3-c2e1-48aa-b9d6-1e39a5f4935e\n92b2f7d5-810c-4cb0-b793-2d8932ded032\na2eaa375-a21e-47cb-8e84-eb430da6c280\n10af3501-3cbb-4735-83e7-db058746efbd\n'

In [ ]:
# Tests for APIKey ls
# Tests using --format flag

with set_airt_service_token_envvar():

    # Testing format flag
    format_str = "{'name': '{}'}"
    result = runner.invoke(app, ["ls", "--format", format_str])
    display(result.stdout)
    key_names = get_values_from_result(result)

    assert result.exit_code == 0
    assert len(key_names) > 0

    # Testing format flag
    format_str = "{'name': '{}', 'uuid': '{}'}"
    result = runner.invoke(app, ["ls", "--format", format_str])
    display(result.stdout)

    assert result.exit_code == 0

    # Testing format flag
    format_str = "{'names': '{}'}"
    result = runner.invoke(app, ["ls", "--format", format_str])
    display(result.stdout)

    assert result.exit_code == 1
    assert "The following columns are not valid" in str(result.stdout)

'RKO4OSLE9DITOFO\n7XIE331PO1OWE71\nDNAK3ICMGDAFQ1K\n8TMEGHX4WXTDGJZ\nYT9B6CT1797HNYP\nSFFJK74O7N32MOR\nIBK7RGHPPG05S4X\nR9JBTBYQF74FSVT\n2BO0JKANPJ46HID\n28YDMEXXMUSANLT\nI9EMJL0SJ919UEX\nJBCQS7CWVT4GM30\nA1N6OAMDI597IYE\n89HJKCDZUSVQQLS\nMF9MJAGCT1SEBKO\n5STC1FGTHT0BB8K\n'

'name             uuid\nRKO4OSLE9DITOFO  76a90be6-4535-4c70-ac2a-16df432913ca\n7XIE331PO1OWE71  35a317ad-0fa1-4e76-ab36-d60a9a31c235\nDNAK3ICMGDAFQ1K  6146ab3a-a49b-4ae3-a628-b6a31583b2d2\n8TMEGHX4WXTDGJZ  3cc3ff1c-374a-4090-bb78-742e5b3aef0a\nYT9B6CT1797HNYP  510edef2-244c-49c8-b8f5-b2c3b500c51b\nSFFJK74O7N32MOR  e4946c54-756e-4e74-9db5-22270bd1c025\nIBK7RGHPPG05S4X  4313caa8-8faa-42ab-8f46-bd6669d96aad\nR9JBTBYQF74FSVT  6ad2a881-bfad-4acb-9d0b-13650e161051\n2BO0JKANPJ46HID  a7bfc801-649e-4574-997f-0dc57bd4e8b5\n28YDMEXXMUSANLT  00d6b61a-62f9-425a-a514-6383b9ffb392\nI9EMJL0SJ919UEX  6e68750d-7c2b-49f4-b2fb-0c3795ec4a32\nJBCQS7CWVT4GM30  78837644-436a-4e4f-82d0-c875d8a72437\nA1N6OAMDI597IYE  835281e3-c2e1-48aa-b9d6-1e39a5f4935e\n89HJKCDZUSVQQLS  92b2f7d5-810c-4cb0-b793-2d8932ded032\nMF9MJAGCT1SEBKO  a2eaa375-a21e-47cb-8e84-eb430da6c280\n5STC1FGTHT0BB8K  10af3501-3cbb-4735-83e7-db058746efbd\n'

"The following columns are not valid: {'names'}. Only the following columns are valid: {'disabled', 'uuid', 'created', 'name', 'expiry'}\nAn example of a valid formatting string: {'uuid': '{}', 'name': '{}', 'created': '{}', 'expiry': '{}', 'disabled': '{}'}\n"

In [ ]:
# Tests for APIKeys ls
# Testing positive scenario.
# Testing by passing different values for  limit


with set_airt_service_token_envvar():

    for limit in [1, 10, 1000]:
        offset = 1
        result = runner.invoke(app, ["ls", "--offset", offset, "--limit", limit, "-q"])

        assert result.exit_code == 0

        ids = get_values_from_result(result)
        display(f"{ids=}")
        assert limit >= len(ids) > 0

"ids=['35a317ad-0fa1-4e76-ab36-d60a9a31c235']"

"ids=['35a317ad-0fa1-4e76-ab36-d60a9a31c235', '6146ab3a-a49b-4ae3-a628-b6a31583b2d2', '3cc3ff1c-374a-4090-bb78-742e5b3aef0a', '510edef2-244c-49c8-b8f5-b2c3b500c51b', 'e4946c54-756e-4e74-9db5-22270bd1c025', '4313caa8-8faa-42ab-8f46-bd6669d96aad', '6ad2a881-bfad-4acb-9d0b-13650e161051', 'a7bfc801-649e-4574-997f-0dc57bd4e8b5', '00d6b61a-62f9-425a-a514-6383b9ffb392', '6e68750d-7c2b-49f4-b2fb-0c3795ec4a32']"

"ids=['35a317ad-0fa1-4e76-ab36-d60a9a31c235', '6146ab3a-a49b-4ae3-a628-b6a31583b2d2', '3cc3ff1c-374a-4090-bb78-742e5b3aef0a', '510edef2-244c-49c8-b8f5-b2c3b500c51b', 'e4946c54-756e-4e74-9db5-22270bd1c025', '4313caa8-8faa-42ab-8f46-bd6669d96aad', '6ad2a881-bfad-4acb-9d0b-13650e161051', 'a7bfc801-649e-4574-997f-0dc57bd4e8b5', '00d6b61a-62f9-425a-a514-6383b9ffb392', '6e68750d-7c2b-49f4-b2fb-0c3795ec4a32', '78837644-436a-4e4f-82d0-c875d8a72437', '835281e3-c2e1-48aa-b9d6-1e39a5f4935e', '92b2f7d5-810c-4cb0-b793-2d8932ded032', 'a2eaa375-a21e-47cb-8e84-eb430da6c280', '10af3501-3cbb-4735-83e7-db058746efbd']"

In [ ]:
# Tests for APIKeys ls
# Testing positive scenario.
# Testing by passing large value for offset.

with set_airt_service_token_envvar():

    limit = 10
    offset = 1_000_000
    result = runner.invoke(app, ["ls", "--offset", offset, "--limit", limit])

    assert result.exit_code == 0

    display(result.stdout)

'uuid    name    created    expiry    disabled\n'

In [ ]:
# Helper context manager for testing

_airt_super_service_token = None


@contextmanager
def set_airt_super_service_token_envvar():
    global _airt_super_service_token
    if _airt_super_service_token is None:
        display("_airt_super_service_token is None, getting a token...")

        username = os.environ["AIRT_SERVICE_SUPER_USER"]
        password = os.environ[SERVICE_PASSWORD]

        Client.get_token(username=username, password=password)
        _airt_super_service_token = Client.auth_token

    try:
        os.environ[SERVICE_TOKEN] = _airt_super_service_token

        yield
    finally:
        del os.environ[SERVICE_TOKEN]


with set_airt_super_service_token_envvar():
    display("*" * len((os.environ[SERVICE_TOKEN])))

'_airt_super_service_token is None, getting a token...'

'*******************************************************************************************************************************'

In [ ]:
# Tests for APIKey ls

with set_airt_service_token_envvar():
    result = runner.invoke(user_cli_app, ["details", "-q"])
    other_user_uuid = result.stdout.replace("\n", "")
    display(other_user_uuid)

    result = runner.invoke(
        user_cli_app,
        ["details", "--user", other_user_uuid, "--format", "{'username': '{}'}"],
    )
    other_user_name = result.stdout[:-1]

# Testing Super user scenarios
with set_airt_super_service_token_envvar():

    # Testing without quite flag
    result = runner.invoke(app, ["ls", "--user", other_user_uuid])
    display(result.stdout)

    assert result.exit_code == 0

    # With quiet
    result = runner.invoke(app, ["ls", "-q", "--user", other_user_name])
    display(result.stdout)

    assert result.exit_code == 0
    ids = get_values_from_result(result)
    display(f"{ids=}")
    assert len(ids) > 0

'd12065d3-48cb-4632-a4cc-db11b843399a'

'uuid                                  name             created         expiry              disabled\n76a90be6-4535-4c70-ac2a-16df432913ca  RKO4OSLE9DITOFO  24 minutes ago  None                False\n35a317ad-0fa1-4e76-ab36-d60a9a31c235  7XIE331PO1OWE71  24 minutes ago  5 months ago        False\n6146ab3a-a49b-4ae3-a628-b6a31583b2d2  DNAK3ICMGDAFQ1K  24 minutes ago  None                False\n3cc3ff1c-374a-4090-bb78-742e5b3aef0a  8TMEGHX4WXTDGJZ  24 minutes ago  None                False\n510edef2-244c-49c8-b8f5-b2c3b500c51b  YT9B6CT1797HNYP  24 minutes ago  None                False\ne4946c54-756e-4e74-9db5-22270bd1c025  SFFJK74O7N32MOR  24 minutes ago  None                False\n4313caa8-8faa-42ab-8f46-bd6669d96aad  IBK7RGHPPG05S4X  24 minutes ago  None                False\n6ad2a881-bfad-4acb-9d0b-13650e161051  R9JBTBYQF74FSVT  24 minutes ago  None                False\na7bfc801-649e-4574-997f-0dc57bd4e8b5  2BO0JKANPJ46HID  24 minutes ago  None                False\n00d6b61a-62f9-42

'76a90be6-4535-4c70-ac2a-16df432913ca\n35a317ad-0fa1-4e76-ab36-d60a9a31c235\n6146ab3a-a49b-4ae3-a628-b6a31583b2d2\n3cc3ff1c-374a-4090-bb78-742e5b3aef0a\n510edef2-244c-49c8-b8f5-b2c3b500c51b\ne4946c54-756e-4e74-9db5-22270bd1c025\n4313caa8-8faa-42ab-8f46-bd6669d96aad\n6ad2a881-bfad-4acb-9d0b-13650e161051\na7bfc801-649e-4574-997f-0dc57bd4e8b5\n00d6b61a-62f9-425a-a514-6383b9ffb392\n6e68750d-7c2b-49f4-b2fb-0c3795ec4a32\n78837644-436a-4e4f-82d0-c875d8a72437\n835281e3-c2e1-48aa-b9d6-1e39a5f4935e\n92b2f7d5-810c-4cb0-b793-2d8932ded032\na2eaa375-a21e-47cb-8e84-eb430da6c280\n10af3501-3cbb-4735-83e7-db058746efbd\n'

"ids=['76a90be6-4535-4c70-ac2a-16df432913ca', '35a317ad-0fa1-4e76-ab36-d60a9a31c235', '6146ab3a-a49b-4ae3-a628-b6a31583b2d2', '3cc3ff1c-374a-4090-bb78-742e5b3aef0a', '510edef2-244c-49c8-b8f5-b2c3b500c51b', 'e4946c54-756e-4e74-9db5-22270bd1c025', '4313caa8-8faa-42ab-8f46-bd6669d96aad', '6ad2a881-bfad-4acb-9d0b-13650e161051', 'a7bfc801-649e-4574-997f-0dc57bd4e8b5', '00d6b61a-62f9-425a-a514-6383b9ffb392', '6e68750d-7c2b-49f4-b2fb-0c3795ec4a32', '78837644-436a-4e4f-82d0-c875d8a72437', '835281e3-c2e1-48aa-b9d6-1e39a5f4935e', '92b2f7d5-810c-4cb0-b793-2d8932ded032', 'a2eaa375-a21e-47cb-8e84-eb430da6c280', '10af3501-3cbb-4735-83e7-db058746efbd']"

In [ ]:
# | exporti


@app.command()
@helper.display_formated_table
@helper.requires_auth_token
def details(
    apikey: str = typer.Argument(
        ...,
        help="ApiKey uuid/name.",
    ),
    format: Optional[str] = typer.Option(
        None,
        "--format",
        "-f",
        help="Format output and show only the given column(s) values.",
    ),
    debug: bool = typer.Option(
        False,
        "--debug",
        "-d",
        help="Set logger level to DEBUG and output everything.",
    ),
) -> Dict["str", Union[pd.DataFrame, str]]:
    """Get the details of an ApiKey."""

    from airt.client import APIKey

    df = APIKey.details(apikey=apikey)

    df["created"] = helper.humanize_date(df["created"])
    df["expiry"] = helper.humanize_date(df["expiry"])

    return {"df": df}

In [ ]:
# tests for APIKey details
with set_airt_service_token_envvar():

    # List the existing API Keys
    format_str = "{'name': '{}'}"
    result = runner.invoke(app, ["ls", "--format", format_str])

    assert result.exit_code == 0
    key_names = get_values_from_result(result)

    # Get details for the first API Key
    first_key = key_names[0]
    result = runner.invoke(app, ["details", first_key])

    display(result.stdout)
    assert result.exit_code == 0

    # Get details for the first API Key
    first_key = key_names[0]
    format_str = "{'name': '{}'}"
    result = runner.invoke(app, ["details", first_key, "--format", format_str])

    display(result.stdout)
    assert result.exit_code == 0

    # Get details for the first API Key
    first_key = key_names[0]
    format_str = "{'name': '{}', 'uuid': '{}'}"
    result = runner.invoke(app, ["details", first_key, "--format", format_str])

    display(result.stdout)
    assert result.exit_code == 0

'uuid                                  name             created         expiry    disabled\n76a90be6-4535-4c70-ac2a-16df432913ca  RKO4OSLE9DITOFO  24 minutes ago  None      False\n'

'RKO4OSLE9DITOFO\n'

'name             uuid\nRKO4OSLE9DITOFO  76a90be6-4535-4c70-ac2a-16df432913ca\n'

In [ ]:
# Tests for details
# Testing negative scenario. Passing invalie API key id

with set_airt_service_token_envvar():

    result = runner.invoke(app, ["details", RANDOM_UUID_FOR_TESTING])

    display(result.stdout)

    assert result.exit_code == 1
    assert "No such apikey" in result.stdout

'Error: No such apikey or not enough authorization to access the apikey.\n'

In [ ]:
# | exporti


@app.command()
@helper.display_formated_table
@helper.requires_totp()
@helper.requires_auth_token
def revoke(
    keys: List[str] = typer.Argument(
        ...,
        help="ApiKey uuid/name to revoke. To revoke multiple keys, please pass the uuids/names separated by space.",
    ),
    user: Optional[str] = typer.Option(
        None,
        "--user",
        help=f"user_uuid/username associated with the ApiKey. To get the user account uuid/username, use the `{CLIENT_NAME} user details` command."
        f" If the user_uuid/username is not passed, then the currently logged-in user_uuid/username will be used.",
    ),
    otp: Optional[str] = typer.Option(
        None,
        "--otp",
        help="Dynamically generated six-digit verification code from the authenticator app. Please pass this optional argument only if the MFA is enabled for your account.",
    ),
    format: Optional[str] = typer.Option(
        None,
        "--format",
        "-f",
        help="Format output and show only the given column(s) values.",
    ),
    quiet: bool = typer.Option(
        False,
        "--quiet",
        "-q",
        help="Output only the revoked ApiKey uuid(s).",
    ),
    debug: bool = typer.Option(
        False,
        "--debug",
        "-d",
        help="Set logger level to DEBUG and output everything.",
    ),
) -> Dict["str", Union[pd.DataFrame, str]]:
    """Revoke one or more ApiKeys"""

    from airt.client import APIKey

    keys = [key for key in keys]
    formated_keys = helper.separate_integers_and_strings(keys)

    df = APIKey.revoke(keys=formated_keys, user=user, otp=otp)  # type: ignore
    df["created"] = helper.humanize_date(df["created"])
    df["expiry"] = helper.humanize_date(df["expiry"])

    return {"df": df}

In [ ]:
# tests for APIKey revoke
# revoking key by name and id

with set_airt_service_token_envvar():

    # List the existing API Keys
    result = runner.invoke(app, ["ls", "-q"])
    assert result.exit_code == 0
    key_ids = get_values_from_result(result)

    # List the existing API Keys
    format_str = "{'name': '{}'}"
    result = runner.invoke(app, ["ls", "--format", format_str])

    assert result.exit_code == 0
    key_names = get_values_from_result(result)

    # Revoking the first API Key
    format_str = "{'name': '{}', 'uuid': '{}'}"
    result = runner.invoke(
        app, ["revoke", key_ids[0], key_names[1], key_names[2], "--format", format_str]
    )

    display(result.stdout)
    assert result.exit_code == 0

'name             uuid\nRKO4OSLE9DITOFO  76a90be6-4535-4c70-ac2a-16df432913ca\n7XIE331PO1OWE71  35a317ad-0fa1-4e76-ab36-d60a9a31c235\nDNAK3ICMGDAFQ1K  6146ab3a-a49b-4ae3-a628-b6a31583b2d2\n'

In [ ]:
# tests for APIKey revoke
with set_airt_service_token_envvar():

    # Create sample keys
    for i in range(10):
        result = runner.invoke(
            app,
            ["create", generate_random_name(), "-e", 365],
        )

    # List the existing API Keys
    result = runner.invoke(app, ["ls", "-q"])

    assert result.exit_code == 0
    ids = get_values_from_result(result)

    # Revoking the first API Key
    first_key = ids[0]
    format_str = "{'name': '{}', 'uuid': '{}'}"
    result = runner.invoke(app, ["revoke", str(first_key), "--format", format_str])
    display(result.stdout)

    assert result.exit_code == 0

    # Running ls command with default settings. The deleted kwy should not be present in the result
    result = runner.invoke(app, ["ls", "-q"])
    ids = get_values_from_result(result)

    display(result.stdout)
    assert result.exit_code == 0
    assert first_key not in ids

    # Running ls command with include_disabled. The deleted kwy should be present in the result
    result = runner.invoke(app, ["ls", "-l", "1000", "-q", "--disabled"])
    ids = get_values_from_result(result)

    display(f"{ids=}")
    display(result.stdout)
    assert result.exit_code == 0
    assert first_key in ids, first_key

'name             uuid\n8TMEGHX4WXTDGJZ  3cc3ff1c-374a-4090-bb78-742e5b3aef0a\n'

'510edef2-244c-49c8-b8f5-b2c3b500c51b\ne4946c54-756e-4e74-9db5-22270bd1c025\n4313caa8-8faa-42ab-8f46-bd6669d96aad\n6ad2a881-bfad-4acb-9d0b-13650e161051\na7bfc801-649e-4574-997f-0dc57bd4e8b5\n00d6b61a-62f9-425a-a514-6383b9ffb392\n6e68750d-7c2b-49f4-b2fb-0c3795ec4a32\n78837644-436a-4e4f-82d0-c875d8a72437\n835281e3-c2e1-48aa-b9d6-1e39a5f4935e\n92b2f7d5-810c-4cb0-b793-2d8932ded032\na2eaa375-a21e-47cb-8e84-eb430da6c280\n10af3501-3cbb-4735-83e7-db058746efbd\n12760028-3a05-47d3-acdc-fb650c6555c9\n804deb8b-8612-41db-a452-cbbfd87ad77d\n435f4338-5584-4ff4-b1ed-c8913b4d12d0\n1ac1b8d5-60f9-4c8f-a90f-050577e52792\n3927ceee-a4cb-43e4-9bdd-c6d607ee5060\nddeb9cb8-de35-4109-ac8e-6f863e3720ec\n06c4a094-5ee3-4c2b-adf4-c5634edf488f\nf1f62f0e-e8ce-4504-ae7d-4e9a8aa4d691\na3555e15-8d40-4b24-8858-622ca60b1e30\n216c8d62-9a3c-495e-b0ab-7edea1d1aff6\n'

"ids=['3d10e7f1-e2cf-4386-9fc5-a32f6af9514f', 'fa6aefe0-e347-40fe-9a40-c3ff203ce3c2', 'db1aad28-f95d-457d-82cb-24aaddc96d56', '7810f834-733f-4189-8c14-bf034eef750b', '8dad6cee-d5de-4409-836a-fc3f0f06cd2a', 'd2eda1ba-5463-49e3-a815-4bc033f02703', '6e4f8e5e-5a43-40e8-baa5-4af7ebfd2333', '9b0cab3d-e932-475b-bb93-74dfec41be87', 'd3c5e334-ce68-4444-ad3c-ed4f2c738a88', '76a90be6-4535-4c70-ac2a-16df432913ca', '35a317ad-0fa1-4e76-ab36-d60a9a31c235', '6146ab3a-a49b-4ae3-a628-b6a31583b2d2', '3cc3ff1c-374a-4090-bb78-742e5b3aef0a', '510edef2-244c-49c8-b8f5-b2c3b500c51b', 'e4946c54-756e-4e74-9db5-22270bd1c025', '4313caa8-8faa-42ab-8f46-bd6669d96aad', '6ad2a881-bfad-4acb-9d0b-13650e161051', 'a7bfc801-649e-4574-997f-0dc57bd4e8b5', '00d6b61a-62f9-425a-a514-6383b9ffb392', '6e68750d-7c2b-49f4-b2fb-0c3795ec4a32', '78837644-436a-4e4f-82d0-c875d8a72437', '835281e3-c2e1-48aa-b9d6-1e39a5f4935e', '92b2f7d5-810c-4cb0-b793-2d8932ded032', 'a2eaa375-a21e-47cb-8e84-eb430da6c280', '10af3501-3cbb-4735-83e7-db058746e

'3d10e7f1-e2cf-4386-9fc5-a32f6af9514f\nfa6aefe0-e347-40fe-9a40-c3ff203ce3c2\ndb1aad28-f95d-457d-82cb-24aaddc96d56\n7810f834-733f-4189-8c14-bf034eef750b\n8dad6cee-d5de-4409-836a-fc3f0f06cd2a\nd2eda1ba-5463-49e3-a815-4bc033f02703\n6e4f8e5e-5a43-40e8-baa5-4af7ebfd2333\n9b0cab3d-e932-475b-bb93-74dfec41be87\nd3c5e334-ce68-4444-ad3c-ed4f2c738a88\n76a90be6-4535-4c70-ac2a-16df432913ca\n35a317ad-0fa1-4e76-ab36-d60a9a31c235\n6146ab3a-a49b-4ae3-a628-b6a31583b2d2\n3cc3ff1c-374a-4090-bb78-742e5b3aef0a\n510edef2-244c-49c8-b8f5-b2c3b500c51b\ne4946c54-756e-4e74-9db5-22270bd1c025\n4313caa8-8faa-42ab-8f46-bd6669d96aad\n6ad2a881-bfad-4acb-9d0b-13650e161051\na7bfc801-649e-4574-997f-0dc57bd4e8b5\n00d6b61a-62f9-425a-a514-6383b9ffb392\n6e68750d-7c2b-49f4-b2fb-0c3795ec4a32\n78837644-436a-4e4f-82d0-c875d8a72437\n835281e3-c2e1-48aa-b9d6-1e39a5f4935e\n92b2f7d5-810c-4cb0-b793-2d8932ded032\na2eaa375-a21e-47cb-8e84-eb430da6c280\n10af3501-3cbb-4735-83e7-db058746efbd\n12760028-3a05-47d3-acdc-fb650c6555c9\n804deb8b-86

In [ ]:
# Tests for revoke
# Testing negative scenario. Passing invalie API key id

with set_airt_service_token_envvar():

    result = runner.invoke(app, ["revoke", RANDOM_UUID_FOR_TESTING])

    display(result.stdout)

    assert result.exit_code == 1
    assert "No such apikey" in result.stdout

    # Testing negative scenario. Passing otp for non-mfa user
    random_otp = 123456
    result = runner.invoke(
        app, ["revoke", RANDOM_UUID_FOR_TESTING, "--otp", random_otp]
    )

    display(result.stdout)
    assert "MFA is not activated for the account" in str(result.stdout)
    assert result.exit_code == 1

'Error: No such apikey or not enough authorization to access the apikey.\n'

'Error: MFA is not activated for the account. Please pass the OTP only after activating the MFA for your account.\n'

In [ ]:
# Super user revoking other user's api_key
with set_airt_service_token_envvar():

    # List the existing API Keys
    result = runner.invoke(app, ["ls", "-q"])

    assert result.exit_code == 0
    ids = get_values_from_result(result)

    # Get details for the first API Key
    expected_key_id = ids[0]

    # get user id
    result = runner.invoke(user_cli_app, ["details", "-q"])
    other_user_uuid = result.stdout.replace("\n", "")


with set_airt_super_service_token_envvar():
    result = runner.invoke(
        app, ["revoke", str(expected_key_id), "--user", other_user_uuid, "-q"]
    )
    display(result.stdout)
    actual_key_id = result.stdout
    assert result.exit_code == 0
    assert actual_key_id == f"{expected_key_id}\n"

'510edef2-244c-49c8-b8f5-b2c3b500c51b\n'

In [ ]:
# revoke multiple keys

with set_airt_service_token_envvar():

    # Creating sample keys
    result = runner.invoke(
        app,
        ["create", generate_random_name(), "-e", 365],
    )
    result = runner.invoke(
        app,
        ["create", generate_random_name(), "-e", 365],
    )
    # List the existing API Keys
    result = runner.invoke(app, ["ls", "-q"])

    assert result.exit_code == 0
    ids = get_values_from_result(result)

    # Get details for the first API Key
    first_key = ids[0]
    second_key = ids[1]
    result = runner.invoke(app, ["revoke", first_key, second_key])
    display(result.output)

    assert result.exit_code == 0
    assert str(first_key) in result.stdout
    assert str(second_key) in result.stdout

'uuid                                  name             created         expiry    disabled\ne4946c54-756e-4e74-9db5-22270bd1c025  SFFJK74O7N32MOR  24 minutes ago  None      True\n4313caa8-8faa-42ab-8f46-bd6669d96aad  IBK7RGHPPG05S4X  24 minutes ago  None      True\n'